In [ ]:
import pandas as pd
import numpy as np
import ast
import nltk
import json
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter

# Read data

In [ ]:
with open('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/categories.txt', 'r') as f:
    categories = f.readlines()
for i in range(len(categories)):
    categories[i] = categories[i].replace('\n', '')
categories

In [ ]:
train_data = pd.read_csv('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/train.csv')
train_data.head(5)

# Preprocess data

In [ ]:
tokenizer = nltk.tokenize.WordPunctTokenizer()
train_data['text'] = train_data['text'].apply(lambda x: ' '.join(tokenizer.tokenize(x)).lower())
train_data.head(5)

In [ ]:
token_counter = Counter()
for text in train_data['text']:
    token_counter.update(text.split(' '))

plt.hist(list(token_counter.values()), range=[0, 3 * 10**3], bins=50, log=True)
plt.xlabel("Word counts")

# Data markup

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "Qwen/Qwen3-4B-Instruct-2507"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def prompt_builder(text):
    prompt = f'''
    Ты — помощник для классификации текстов.  
    Твоя задача: по тексту отзыва определить, к какой из категорий он относится.  

    Доступные категории:  
    1. бытовая техника - холодильники, стиральные машины, плиты, микроволновки, чайники, пылесосы и другая техника для дома.
    2. обувь - кроссовки, ботинки, туфли, сандалии, сапоги и другая обувь для взрослых и детей.  
    3. одежда - платья, брюки, юбки, футболки, свитеры, куртки, бельё, бюстгальтеры, верхняя одежда.
    4. посуда - только тарелки, кружки, чашки для еды, столовые приборы и кухонная утварь; НЕ включает одежду, бельё или чашки бюстгальтеров.  
    5. текстиль - только постельное бельё, покрывала, наволочки, кухонные полотенца, пледы, портьеры, шторы, ковры, халаты и так далее. 
    6. товары для детей -  игрушки, детская одежда, детская мебель, товары для ухода за детьми, коляски, автокресла. 
    7. украшения и аксессуары - серьги, кольца, браслеты, ожерелья, ремни, шарфы, сумки, очки и другие модные аксессуары.
    8. электроника - смартфоны, планшеты, ноутбуки, камеры, наушники, колонки, смарт-часы и прочая электроника. 
    9. нет товара - отзыв не содержит упоминания конкретного товара.

    Жёсткие правила:  
    - Выбирай строго одну из доступных категорий.  
    - Если отзыв не содержит упоминания товара — «нет товара».  
    - Слово «чашка» проверяй по контексту: если речь о бюстгальтере/одежде → «одежда», если речь о посуде → «посуда».  
    - Не добавляй новые категории и не исправляй написание слов.  
    - Отвечай строго в формате JSON:  
    {{"category": "<название категории>"}}  

    Примеры:  
    Отзыв: "Купила новые кроссовки, очень удобные."  
    Ответ: {{"category": "обувь"}}  
    
    Отзыв: "Доставка пришла вовремя, но сервис ужасный."  
    Ответ: {{"category": "нет товара"}}  
    
    Отзыв: "Купила постельное бельё из хлопка, качество хорошее."  
    Ответ: {{"category": "текстиль"}}  
    
    Отзыв: "Чашка бюстгальтера слишком мала, размер не подошёл."  
    Ответ: {{"category": "одежда"}}  
    
    Отзыв: "Купила новые кружки для чая, качество отличное."  
    Ответ: {{"category": "посуда"}}  
    
    Теперь классифицируй следующий отзыв:  
    <{text}>
    '''
    
    return prompt


def get_llm_answer(prompt, max_tokens=512, temperature=0.0, top_p=1.0, do_sample=False):
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p
    )
    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
    response = tokenizer.decode(output_ids, skip_special_tokens=True)

    return response

In [ ]:
result = {'text': [], 'category': []}
for text in tqdm(train_data['text'].values, desc='Разметка данных', total=len(train_data)):
    result['text'].append(text)
    try:
        result['category'].append(json.loads(get_llm_answer(prompt_builder(text))['category']))
    except:
        print('!')
        result['category'].append("no category")

In [ ]:
marked_data = pd.DataFrame(result)
marked_data

In [ ]:
marked_data.to_csv('marked_data_v3.csv', index=False)

# Postprocessing of marked up data

In [ ]:
marked_data = pd.read_csv('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/marked_data_v3.csv')
marked_data.head(5)

In [ ]:
marked_data['category'].value_counts()

In [ ]:
# товары почеченные как "детская одежда" можно отнести с категории "товары для детей"
print(marked_data[marked_data['category'] == 'детская одежда']['text'].values)

In [ ]:
# не смотря на инструкции в промпте в категорию "посуда" попали отзывы которые относятся к бюстгальтерам
print(marked_data[marked_data['category'] == 'посуда']['text'].values)

In [ ]:
marked_data = marked_data[marked_data['category'] != 'no category']

In [ ]:
mapping = {}
for cat in categories:
    mapping[cat] = cat
mapping[' tekstil'] = 'текстиль'
mapping['детская одежда'] = 'товары для детей'
mapping['посуда'] = 'одежда'

marked_data['category'] = marked_data['category'].apply(lambda x: mapping[x])
marked_data['category'].value_counts()

# Synthetic data generation

### выбирем лучших представителей своих категорий для помощи в генерации

In [ ]:
best_goods_for_kids_examples = [
    'х / б носочки , хорошее качество , пришли вовремя , дочь очень довольна .',
    'размер не соответсвует , немного не тот цвет , рассчитан для девочек лет 10 - 12',
    'кофточка оказалось очень маленькой . пришлась в пору моей 5 - ти летней племяннице .',
    'шапку покупала ребёнку из за пампона ( он чёрный ) так вот он совсем не держится на шапке !!! продавцу написала об этом но он игнорит !',
    'шапка маленькая , на ребенка',
    'заказали размер м на 46 , оказалось мало . у них м как как у нас xs ( 42 ). куртка для детей , нет выточки под грудь .',
    'отличная шапка , бубон натуральный енот - супер , пришла в брянск за 2 недели',
    'заказывала s а пришло далеко не s на девочку лет 10 - 11',
    'быстрая доставка , но маловато . xxl на девочку 46 размера в обтяг .',    
]

best_shoes_examples = [
    'маленькие . когда одеваешь рисунка почти нет . белая подошва .',
    'рисунок только до половины носка , то есть только сверху , нижняя часть стопы однотонным белым цветом . некрасиво выглядит переход от рисунка к белому цвету .',
    'получила через месяц в питер . качество хорошее , у меня р - р 35 - 36 мне как раз . не знаю как их натягивают на 37 - 38',
    'товар не понравился . у меня не очень крупные ноги и в комментариях я прочитала что они тянутся , размер всегда носила s , в общем и заказала s , оказались очень велики , я не довольна покупкой .',
    'маленькие , размер не соответствует',
    'размер не соответствует ((( заказывала жве пары до этого , все было размер в размер'
]

best_jewelry_and_accessories_examples = [
    'класс ! мне понравились , очень красивые , стразы держатся крепко !',
    'у очков в белой оправе было деформировано стекло , все как будто " плывет "',
    'продавец слукавил . материал шарфв не является обещанным кашемиром .',
    'заказала шарф 13 . 10 , а пришёл ко мне в крым он уже 28 . 10 . доставка быстрая ! шарф просто любовь с первого взгляда и прикосновения . очень мягкий , тёплый и большой . цвета яркие , как на фото ',
]

best_tekstil_examples = [
    'отвратительное качество материала . продавец обещает хлопок , а пришла голая синтетика . сильный запах .',
    'какой - то тонкий синтетический халатик + отправка очень долгая .',
    'хороший халат , правда думала ткань плотнее будет просвечивает .',
    'ткань как дешёвый трикотаж в смешных ценах ))',
    'доставка по времени норм , недели 3 вроде , но ткань .. ужасная просто , синтетика , к телу неприятно колется , фу много конечно не ждала , но надеялась что будет хоть чуть лучше',
    'в целом , конечно , неплохой халат . да за такие деньги чуда и не ожидалось . но хотелось бы , чтобы он не приходил хотя бы рваным .'
]

In [ ]:
def generation_prompt_builder(category_name, category_description, num_examples, seed_examples=None,
                              style=None,
                              meta_info='''Отзывы в стиле маркетплейса, 1–3 коротких предложения, допустимы лёгкие орфографические ошибки. 
                              Старайся, чтобы каждый отзыв отличался формулировками, упоминай разные детали: 
                              запах, внешний вид, удобство, упаковку, срок службы, соответствие фото, цену.'''):
    prompt = f'''
Сгенерируй {num_examples} реалистичных отзывов на русском языке
Для категории {category_name}
Описание категории: {category_description}
{meta_info}
{style}\n
'''

    if seed_examples:
        examples = '\n'.join([f'{i + 1}. {ex}' for i, ex in enumerate(seed_examples)])
        prompt += 'Вот примеры реальных отзывов:\n'
        prompt += examples + '\n'
    else:
        prompt += "У тебя нет готовых примеров. Придумай отзывы самостоятельно на основе описания категории."

    prompt += '\n\nОтвет верни строго в формате JSON-массива строк, например:\n["отзыв_1", "отзыв_2"]'

    return prompt

In [ ]:
styles = [
    'Сделай отзывы радостными, покупатель доволен, эмоции счастья и удовлетворения от товара',
    'Сделай отзывы критическими, покупатель разочарован, укажи недостатки товара или проблемы с размером, качеством, доставкой',
    'Отзывы нейтральные, описательные, без сильной эмоции, упор на детали товара, доставки и упаковки',
    'Добавь упоминание упаковки, цвета, размера, сроков доставки',
    'Очень краткие отзывы, 1–2 предложения, просто и понятно'
]

category_info = {
    'товары для детей': {
        'desc': 'игрушки, детская одежда, детская мебель, товары для ухода за детьми, коляски, автокресла.',
        'examples': best_goods_for_kids_examples
    },
    'украшения и аксессуары': {
        'desc': 'серьги, кольца, браслеты, ожерелья, ремни, шарфы, сумки, очки и другие модные аксессуары.',
        'examples': best_jewelry_and_accessories_examples
    },
    'обувь': {
        'desc': 'кроссовки, ботинки, туфли, сандалии, сапоги и другая обувь для взрослых и детей.',
        'examples': best_shoes_examples
    },
    'текстиль': {
        'desc': 'только постельное бельё, покрывала, наволочки, кухонные полотенца, пледы, портьеры, шторы, ковры, халаты и так далее.',
        'examples': best_tekstil_examples
    }
}
no_examples_category_info = {
    'посуда': 'только тарелки, кружки, чашки для еды, столовые приборы и кухонная утварь; НЕ включает одежду, бельё или чашки бюстгальтеров.',
    'бытовая техника': 'холодильники, стиральные машины, плиты, микроволновки, чайники, пылесосы и другая техника для дома.',
    'электроника': 'смартфоны, планшеты, ноутбуки, камеры, наушники, колонки, смарт-часы и прочая электроника.'
}

In [ ]:
batch_size = 5
result = {
    'товары для детей': [],
    'украшения и аксессуары': [],
    'обувь': [],
    'текстиль': []
}
for cat in category_info.keys():
    for i in tqdm(range(60), desc=f'Генерация для категории: {cat}', total=60):
        try:
            result[cat].append(get_llm_answer(generation_prompt_builder(
                cat,
                category_info[cat]['desc'],
                batch_size,
                category_info[cat]['examples'],
                random.choice(styles),
            ), 4096, 0.7, 0.9, True))
        except:
            print(f'Something went wrong, category: {cat}')

In [ ]:
def make_set(category):
    result_set = set()
    for batch in result[category]:
        try:
            result_set.update(json.loads(batch))
        except:
            print(batch)
    return list(result_set)

In [ ]:
goods_for_kids = make_set('товары для детей')
goods_for_kids = pd.DataFrame({'text': goods_for_kids, 'category': 'товары для детей'})
jewelry_and_accessories = make_set('украшения и аксессуары')
jewelry_and_accessories = pd.DataFrame({'text': jewelry_and_accessories, 'category': 'украшения и аксессуары'})
shoes = make_set('обувь')
shoes = pd.DataFrame({'text': shoes, 'category': 'обувь'})
tekstil = make_set('текстиль')
tekstil = pd.DataFrame({'text': tekstil, 'category': 'текстиль'})

generated_data = pd.concat([goods_for_kids, jewelry_and_accessories, shoes, tekstil], ignore_index=True)

In [ ]:
generated_data['category'].value_counts()

In [ ]:
generated_data.to_csv('generated_data.csv', index=False)

In [ ]:
batch_size = 5
result = {
    'посуда': [],
    'бытовая техника': [],
    'электроника': []
}
for cat in no_examples_category_info.keys():
    for i in tqdm(range(60), desc=f'Генерация для категории: {cat}', total=60):
        try:
            result[cat].append(get_llm_answer(generation_prompt_builder(
                cat,
                no_examples_category_info[cat],
                batch_size,
                None,
                random.choice(styles),
            ), 4096, 0.7, 0.9, True))
        except:
            print(f'Something went wrong, category: {cat}')

In [ ]:
dfs = []
for cat in no_examples_category_info.keys():
    dfs.append(pd.DataFrame({'text': make_set(cat), 'category': cat}))
no_examples_generated_data = pd.concat([df for df in dfs], ignore_index=True)

In [ ]:
no_examples_generated_data['category'].value_counts()

In [ ]:
no_examples_generated_data.to_csv('no_examples_generated_data.csv', index=False)

# Classification task

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from datasets import Dataset
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
generated_data = pd.read_csv('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/generated_data.csv')
no_ex_generated_data = pd.read_csv('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/no_examples_generated_data.csv')
data = pd.concat([marked_data, generated_data, no_ex_generated_data], ignore_index=True)
data['category'].value_counts()

In [ ]:
tokenizer = nltk.tokenize.WordPunctTokenizer()
data['text'] = data['text'].apply(lambda x: ' '.join(tokenizer.tokenize(x)).lower())
data.head(5)

In [ ]:
token_counter = Counter()
for text in data['text']:
    token_counter.update(text.split(' '))

plt.hist(list(token_counter.values()), range=[0, 3 * 10**3], bins=50, log=True)
plt.xlabel("Word counts")

In [ ]:
X = data['text'].values
y = data['category'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
train_dataset = Dataset.from_dict({'text': X_train, 'category': y_train})
test_dataset = Dataset.from_dict({'text': X_test, 'category': y_test})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruRoberta-large')
data_collator = DataCollatorWithPadding(tokenizer)
model = AutoModelForSequenceClassification.from_pretrained('ai-forever/ruRoberta-large', num_labels=len(categories))

In [ ]:
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    
    eval_steps=50,
    eval_strategy='steps',

    save_strategy='steps',
    save_steps=50,
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model='weighted_f1',
    greater_is_better=True,
    
    logging_steps=50,
    report_to='none',
)

In [ ]:
def tokenize_fn(batch):
    return tokenizer(batch['text'], padding=False)

train_dataset = train_dataset.map(tokenize_fn, batched=True).rename_column('category', 'labels').remove_columns(['text'])
test_dataset = test_dataset.map(tokenize_fn, batched=True).rename_column('category', 'labels').remove_columns(['text'])

In [ ]:
def encode_labels(batch):
    mapping = {cat: i for i, cat in enumerate(categories)}
    batch['labels'] = [mapping[label] for label in batch['labels']]
    return batch

train_dataset = train_dataset.map(encode_labels, batched=True)
test_dataset = test_dataset.map(encode_labels, batched=True)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {'weighted_f1': f1_score(labels, preds, average='weighted')}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
train_loss = []
eval_loss = []
grad_norm = []
for item in trainer.state.log_history:
    if item.get('loss', None):
        train_loss.append(item['loss'])
    if item.get('eval_loss', None):
        eval_loss.append(item['eval_loss'])
    if item.get('grad_norm', None):
        grad_norm.append(item['grad_norm'])
steps = [(i + 1) * 50 for i in range(len(train_loss))]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].plot(steps, train_loss, label='Train Loss', color='blue')
axes[0].plot(steps, eval_loss, label='Eval Loss', color='orange')
axes[0].set_title("Train Loss vs Eval Loss")
axes[0].set_xlabel("Step")
axes[0].set_ylabel("Loss")
axes[0].legend()

axes[1].plot(steps, grad_norm, label='Grad Norm', color='green')
axes[1].set_title('Gradient Norm')
axes[1].set_xlabel('Step')
axes[1].set_ylabel('Grad Norm')
axes[1].legend()

plt.tight_layout()
plt.show()

# Make predictions

In [ ]:
example = pd.read_csv('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/submission_example.csv')
example.sample(5)

In [ ]:
test_data = pd.read_csv('/kaggle/input/tinkoff-ml-sirius/tinkoff_ml_sirius/test.csv')
test_data.sample(5)

In [ ]:
tokenizer = nltk.tokenize.WordPunctTokenizer()
test_data['text'] = test_data['text'].apply(lambda x: ' '.join(tokenizer.tokenize(x)).lower())
test_dataset = Dataset.from_dict({'text': test_data['text'].values})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai-forever/ruRoberta-large')
def tokenize_fn(batch):
    return tokenizer(batch['text'], padding=False)

test_dataset = test_dataset.map(tokenize_fn, batched=True).remove_columns(['text'])

In [ ]:
preds = trainer.predict(test_dataset)
mapping = {i: cat for i, cat in enumerate(categories)}
result = [mapping[np.argmax(pred)] for pred in preds.predictions]

In [ ]:
print(f"Кол-во обрабатываемых отзывов в секунду: {preds.metrics['test_samples_per_second']}")

### Кол-во обрабатываемых отзывов в секунду: 185.835. Инференс модели укладывается в заданные ограничения - не более 5 секунд на классификацию одного примера.

In [ ]:
result_df = pd.DataFrame({'category': result})
result_df.sample(5)

In [ ]:
result_df.to_csv('submission.csv', index=False)